In [1]:
import pandas as pd

In [61]:
answer_key = pd.read_csv(r'dev\annotator-assessment-profiler\answer\answer_key.csv',sep='\t')
responses = pd.read_csv(r'dev\annotator-assessment-profiler\response\YTL AI Labs Annotator Application & Assessment(Sheet1).csv',header=[0,1])

In [44]:
# Create a new, single-level list of column names by joining the two levels
# Example: ('Malay Q1 (Prompt)', 'Evaluate whether...') -> 'Malay Q1 (Prompt) | Evaluate whether...'
responses.columns = [' | '.join(col).strip() for col in responses.columns.values]

In [46]:
all_columns = responses.columns.tolist()
prefixes = ("Malay", "English", "Chinese")

# 3. Use list comprehension to filter the columns
language_columns = [
    col for col in all_columns
    if col.startswith(prefixes)
]

In [47]:
import re

# Define the full text patterns you want to hard-rename
PROMPT_STATUS_PATTERN = "evaluate whether the prompt.*is usable.*"
RESPONSE_STATUS_PATTERN = "evaluate the quality of the response.*in the bolded blue text below.*"

# The dictionary to store the final mapping
column_map = {}

for old_col in responses.columns:
    # 1. Lowercase the entire column for consistent matching
    lower_col = old_col.lower().strip()

    # 2. Split the column into the prefix and the evaluation question
    # Example: 'malay q1 (prompt) | evaluate...'
    parts = lower_col.split(' | ', 1)
    if len(parts) < 2:
        # Skip columns that don't match the expected MultiIndex format
        continue

    prefix, question = parts[0], parts[1]

    # Clean up the prefix: 'malay q1 (prompt)' -> 'malay_q1_prompt'
    # Use re.sub to remove non-alphanumeric/non-space characters, then replace spaces with underscores
    cleaned_prefix = re.sub(r'[^a-z0-9\s]', '', prefix).replace(' ', '_').strip('_')
    
    # 3. Apply the mapping logic to determine the suffix
    new_suffix = None
    if re.search(PROMPT_STATUS_PATTERN, question):
        new_suffix = "status"
    elif re.search(RESPONSE_STATUS_PATTERN, question):
        new_suffix = "status"
    
    # If a new suffix was found, construct the new column name
    if new_suffix:
        new_col_name = f"{cleaned_prefix}_{new_suffix}"
        
        # 4. Add the original column (case-sensitive) and the new name to the map
        column_map[old_col] = new_col_name
    
# Print the final mapping dictionary
print("### Generated Column Rename Map ###")
print("column_map = {")
for old, new in column_map.items():
    print(f'    "{old}": "{new}",')
print("}")

# Now you can apply this to your DataFrame:
responses.rename(columns=column_map, inplace=True)

### Generated Column Rename Map ###
column_map = {
    "Malay Q1 (Prompt) | Evaluate whether the prompt in the image above OR in the bolded blue text below is usable.": "malay_q1_prompt_status",
    "Malay Q1 (Response) | Evaluate the quality of the response in the image above OR in the bolded blue text below.": "malay_q1_response_status",
    "Malay Q2 (prompt) | Evaluate whether the prompt written in the image above OR in the bolded blue text below is usable.": "malay_q2_prompt_status",
    "Malay Q2 (response) | Evaluate the quality of the response written in the image above OR in the bolded blue text below.": "malay_q2_response_status",
    "Malay Q3 (prompt) | Evaluate whether the prompt written in the image above OR in the bolded blue text below is usable.": "malay_q3_prompt_status",
    "Malay Q3 (response) | Evaluate the quality of the response written in the image above OR in the bolded blue text below": "malay_q3_response_status",
    "Malay Q4 (prompt) | Evaluate whether the

In [48]:
import numpy as np

# 1. Define the specific hard rename map for the CONFIDENCE_COLUMNS
CONFIDENT_LANGUAGE_MAP = {
    'Unnamed: 15_level_0 | This question is about Bahasa Melayu (BM) language abilities.\nAre you confident reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language': 'malay_confidence',
    'Unnamed: 16_level_0 | This question is about English language abilities.\nAre you confident reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language (e.g. news': 'english_confidence',
    'Unnamed: 17_level_0 | This question is about Mandarin Chinese language abilities.\nAre you comfortable reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language': 'chinese_confidence'
}

# 2. Define the function for value encoding
def encode_confidence(value):
    """Encodes the string value to 0 if 'no' is present, or 1 otherwise."""
    if pd.isna(value):
        return np.nan
    # Convert to string and check if 'no' is in the lowercased value
    if 'no' in str(value).lower():
        return 0
    return 1

# 3. Define the main function to load, rename, and process the data
def process_confidence_columns(df):

    df.rename(columns=CONFIDENT_LANGUAGE_MAP, inplace=True)

    new_confidence_cols = list(CONFIDENT_LANGUAGE_MAP.values())

    for col in new_confidence_cols:
        if col in df.columns:
            df[col] = df[col].apply(encode_confidence)

    print(df[new_confidence_cols].head())
    print(df[new_confidence_cols].info())
    
    return df

responses2 = process_confidence_columns(responses)

   malay_confidence  english_confidence  chinese_confidence
0               1.0                 1.0                 0.0
1               1.0                 1.0                 1.0
2               1.0                 1.0                 1.0
3               1.0                 1.0                 1.0
4               1.0                 1.0                 1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   malay_confidence    136 non-null    float64
 1   english_confidence  136 non-null    float64
 2   chinese_confidence  136 non-null    float64
dtypes: float64(3)
memory usage: 3.4 KB
None


In [49]:
ID_COLUMNS_MAP = {'Unnamed: 6_level_0 | Full name (according to NRIC):':"name",
                  'Unnamed: 9_level_0 | IC number:':"ic_number"}

responses2.rename(columns=ID_COLUMNS_MAP,inplace=True)

responses2.head(2)

responses3 = responses2.copy()

In [51]:
renamed_scoring_columns = [
    col for col in responses3.columns
    if col.startswith(("malay_","chinese_","english_"))
]

scoring_columns = ['ic_number','name']+ renamed_scoring_columns
responses3 = responses3[scoring_columns]
responses3.head(2)

,ic_number,name,malay_confidence,english_confidence,chinese_confidence,malay_q1_prompt_status,malay_q1_response_status,malay_q2_prompt_status,malay_q2_response_status,malay_q3_prompt_status,malay_q3_response_status,malay_q4_prompt_status,malay_q4_response_status,malay_q5_prompt_status,malay_q5_response_status,english_q1_prompt_status,english_q1_response_status,english_q2_prompt_status,english_q2_response_status,english_q3_prompt_status,english_q3_response_status,english_q4_prompt_status,english_q4_response_status,english_q5_prompt_status,english_q5_response_status,chinese_q1_prompt_status,chinese_q1_response_status,chinese_q2_prompt_status,chinese_q2_response_status,chinese_q3_prompt_status,chinese_q3_response_status,chinese_q4_prompt_status,chinese_q4_response_status,chinese_q5_prompt_status,chinese_q5_response_status
0,6.051108e+10,Tang Eugine,1.0,1.0,0.0,"FAIL (Minor rewrite, ≤ 2 sentences)",PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),PASS,PASS (no changes),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.011108e+10,Tang Yvone,1.0,1.0,1.0,"FAIL (Major rewrite, > 2 sentences)",PASS (no changes),PASS,FAIL (needs improvement),PASS,PASS (no changes),PASS,PASS (no changes),FAIL (MINOR REWRITE),PASS (no changes),PASS,FAIL (major issues),PASS,FAIL (major issues),FAIL (MINOR REWRITE),PASS (no changes),NOT USABLE,FAIL (needs improvement),FAIL (MINOR REWRITE),PASS (no changes),"FAIL (Minor Rewrite, ≤ 2 sentences)",PASS (no changes),PASS,FAIL (needs improvement),PASS,PASS (no changes),PASS,FAIL (needs improvement),PASS,PASS (no changes)


In [54]:
responses3.columns

Index(['ic_number', 'name', 'malay_confidence', 'english_confidence',
       'chinese_confidence', 'malay_q1_prompt_status',
       'malay_q1_response_status', 'malay_q2_prompt_status',
       'malay_q2_response_status', 'malay_q3_prompt_status',
       'malay_q3_response_status', 'malay_q4_prompt_status',
       'malay_q4_response_status', 'malay_q5_prompt_status',
       'malay_q5_response_status', 'english_q1_prompt_status',
       'english_q1_response_status', 'english_q2_prompt_status',
       'english_q2_response_status', 'english_q3_prompt_status',
       'english_q3_response_status', 'english_q4_prompt_status',
       'english_q4_response_status', 'english_q5_prompt_status',
       'english_q5_response_status', 'chinese_q1_prompt_status',
       'chinese_q1_response_status', 'chinese_q2_prompt_status',
       'chinese_q2_response_status', 'chinese_q3_prompt_status',
       'chinese_q3_response_status', 'chinese_q4_prompt_status',
       'chinese_q4_response_status', 'chinese_q5

In [65]:
import pandas as pd
import numpy as np
import re

def clean_answer(ans):
    """Standardizes the answer string to 'PASS', 'FAIL', or None (for NaN/other)."""
    if pd.isna(ans) or ans is None:
        return None
    
    # Convert to string and uppercase to handle case variations
    ans_str = str(ans).upper().strip()
    
    # Use regex to find 'PASS' or 'FAIL' anywhere in the string
    if re.search(r'PASS', ans_str):
        return 'PASS'
    elif re.search(r'FAIL', ans_str):
        return 'FAIL'
        
    return None

def score_match(response_ans, key_ans):
    """
    Applies the custom scoring logic for a single question response:
    1. key=FAIL and response=PASS => 0.0
    2. key=PASS and response=FAIL => 0.5
    3. Matched => 1.0
    4. NaN in key or response => NaN
    """
    clean_response = clean_answer(response_ans)
    clean_key = clean_answer(key_ans)

    # Rule 4: Cannot score if either is missing or unreadable
    if clean_response is None or clean_key is None:
        return np.nan 

    # Rule 3: Perfect match
    if clean_key == clean_response:
        return 1.0 
    
    # Mismatched Cases
    if clean_key == 'FAIL' and clean_response == 'PASS':
        # Rule 1: Key says FAIL, Response says PASS => 0.0 (Strict mismatch)
        return 0.0
    elif clean_key == 'PASS' and clean_response == 'FAIL':
        # Rule 2: Key says PASS, Response says FAIL => 0.5 (Partial credit)
        return 0.5
    
    # Fallback for unexpected clean combinations (e.g. both are 'OTHER')
    return np.nan

def calculate_all_scores(responses_df, answer_key_df):
    """Calculates individual question scores and sums them up by language."""
    
    # 1. Prepare Answer Key
    # Ensure the key is clean and ready for comparison
    answer_key_df['clean_answer'] = answer_key_df['answer'].apply(clean_answer)

    # 2. Iterate through the Answer Key and Score each question in responses_df
    for _, row in answer_key_df.iterrows():
        q_key = row['question_key']
        clean_key = row['clean_answer']
        
        # Check if the column exists in the responses DataFrame
        if q_key in responses_df.columns:
            score_col_name = f"{q_key}_score"
            
            # Apply the scoring logic row-wise
            responses_df[score_col_name] = responses_df[q_key].apply(
                lambda resp_ans: score_match(resp_ans, clean_key)
            )

    # 3. Sum the scores by language
    
    # Identify all the newly created score columns
    all_score_cols = [col for col in responses_df.columns if col.endswith('_score')]
    
    # Define language groups
    lang_prefixes = ['malay_', 'english_', 'chinese_']
    
    final_score_cols = []
    for prefix in lang_prefixes:
        # Find all score columns for the current language
        lang_score_cols = [col for col in all_score_cols if col.startswith(prefix)]
        
        new_score_col_name = f'{prefix}scores'
        
        if lang_score_cols:
            # Sum the individual question scores for the final language score
            responses_df[new_score_col_name] = responses_df[lang_score_cols].sum(axis=1)
        else:
            # If no questions were scored for a language, set score to 0
            responses_df[new_score_col_name] = 0.0 
            
        final_score_cols.append(new_score_col_name)
            
    # Return the columns with the final summed scores
    return responses_df[['ic_number', 'name','malay_confidence','english_confidence','chinese_confidence'] + final_score_cols]

# --- Execution Block ---
# You need to run this part after loading your data.
final_scores_df = calculate_all_scores(responses3, answer_key)
final_scores_df

,ic_number,name,malay_confidence,english_confidence,chinese_confidence,malay_scores,english_scores,chinese_scores
0,6.051108e+10,Tang Eugine,1.0,1.0,0.0,5.0,3.0,0.0
1,5.011108e+10,Tang Yvone,1.0,1.0,1.0,7.0,4.5,7.0
2,5.062514e+10,KHONG YIROU,1.0,1.0,1.0,6.5,5.0,6.5
3,5.060905e+10,SIA ZI YUE,1.0,1.0,1.0,7.0,5.0,5.0
4,5.021101e+10,OI KAY YI,1.0,1.0,1.0,6.0,6.0,7.5
5,9.109210e+11,Toh Jyh Yee,0.0,1.0,1.0,0.0,3.0,4.0
6,9.905040e+11,Mohamad Suhairi bin Md Subhi,1.0,1.0,0.0,8.0,6.0,0.0
7,9.604300e+11,Lee Siew Hwei,1.0,1.0,1.0,5.0,5.0,6.5
8,9.712280e+11,Edward Lim Heng Yuan,1.0,1.0,0.0,7.0,5.0,0.0
9,7.601070e+11,MOHD DIAH BIN A.KARIM,1.0,1.0,0.0,7.0,4.0,0.0
